# Kernel Methods challenge

Importing base libraries...

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## Debugging requirements
import pdb

## Performance metrics requirements
import time

## Kernel SVM requirements
from cvxopt import matrix
from cvxopt import solvers
from scipy.spatial.distance import cdist
from numpy.core.defchararray import not_equal

## 1.Loading the data + sanity checks

In [ ]:
def load_data(dsID, set_type='tr', folder_name='data'):
    Xdata_file = folder_name + '/X' + set_type + str(dsID) + '.csv'
    X = pd.read_csv(Xdata_file, header=None, names=['Sequence'], dtype={'Sequence': np.unicode_})
    if set_type=='tr':
        Ydata_file = folder_name + '/Y' + set_type + str(dsID) + '.csv'
        Y = pd.read_csv(Ydata_file, index_col=0, dtype={'Bound': np.dtype(bool)})
        Y.index = Y.index - 1000*dsID
        df = pd.concat([X, Y], axis=1)
    else:
        df = X
    return df

## Loading training data
tr0 = load_data(0, 'tr')
tr1 = load_data(1, 'tr')
tr2 = load_data(2, 'tr')

## Loading test data
te0 = load_data(0, 'te')
te1 = load_data(1, 'te')
te2 = load_data(2, 'te')

Some sanity checks...

<b>Training set 0</b>

In [ ]:
tr0['Bound'].describe()

In [ ]:
tr0.head(5)

In [ ]:
tr0.tail(5)

<b>Training set 1</b>

In [ ]:
tr1['Bound'].describe()

In [ ]:
tr1.head(5)

In [ ]:
tr1.tail(5)

<b>Training set 2</b>

In [ ]:
tr2['Bound'].describe()

In [ ]:
tr2.head(5)

In [ ]:
tr2.tail(5)

<b>Test set 0</b>

In [ ]:
te0['Sequence'].describe()

In [ ]:
te0.head(5)

In [ ]:
te0.tail(5)

<b>Test set 1</b>

In [ ]:
te1['Sequence'].describe()

In [ ]:
te1.head(5)

In [ ]:
te1.tail(5)

<b>Test set 2</b>

In [ ]:
te2['Sequence'].describe()

In [ ]:
te2.head(5)

In [ ]:
te2.tail(5)

First idea: use some distance on the strings as a kernel.
However, note that some distances (Hamming) are only defined for sequences of the same size.
What is the mininimum and maximum length of the DNA sequences in this first train set?

In [ ]:
min_length = tr0['Sequence'].str.len().max(0)
max_length = tr0['Sequence'].str.len().max(0)

In [ ]:
print('Min sequence length: {}'.format(min_length))
print('Max sequence length: {}'.format(max_length))
print('Length amplitude: {}'.format(max_length-min_length))

## 2. Defining first kernels + running simple classification model

### First kernels

Ok, so here all sequences have the same length. That means that we can start by something simple like Hamming. However, we may want to use something that would seamlessly extend to DNA sequences of different lengths...
Here I will test both the Hamming and the Levenshtein distance as kernels for mapping DNA sequences:

In [ ]:
## Defining both string distances for first kernel tryouts:

def hamming_distance(source, target):
    """Return the Hamming distance between equal-length sequences"""
    if len(source) != len(target):
        raise ValueError("Undefined for sequences of unequal length")
    return np.count_nonzero(not_equal(source,target))

def levenshtein_distance(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]


def build_kernel(arr1, arr2, kernel_fct):
    """Builds the kernel matrix from numpy array @arr and kernel function @kernel_fct. V1, unnefficient"""
    try:
        assert len(arr1) > 0
        assert len(arr2) > 0
    except AssertionError:
        print('At least one of the argument arrays is empty')
    if arr1.ndim == 1:
        arr1 = arr1.reshape((len(arr1),1))
    if arr2.ndim == 1:
        arr2 = arr2.reshape((len(arr2),1))
    K = cdist(arr1, arr2, lambda u, v: kernel_fct(list(u[0]),list(v[0])))
    return K

Testing kernel computation speed (debugging only):

In [ ]:
t0 = time.time()
Ktr0 = build_kernel(tr0['Sequence'], tr0['Sequence'], kernel_fct = hamming_distance)
t1 = time.time()
Ktr1 = build_kernel(tr1['Sequence'], tr1['Sequence'], kernel_fct = hamming_distance)
t2 = time.time()
Ktr2 = build_kernel(tr2['Sequence'], tr2['Sequence'], kernel_fct = hamming_distance)
t3 = time.time()

In [ ]:
print('Preparing kernel matrix for a training dataset 1 took {0:d}min {1:d}s with this method'.format(int((t1-t0)/60),int(t1-t0)%60))
print('Preparing kernel matrix for a training dataset 2 took {0:d}min {1:d}s with this method'.format(int((t2-t1)/60),int(t2-t1)%60))
print('Preparing kernel matrix for a training dataset 3 took {0:d}min {1:d}s with this method'.format(int((t3-t2)/60),int(t3-t2)%60))

### Tools

Defining a couple of losses functions that will be useful:

In [ ]:
## Squared loss
def ls_squared(preds, labels):
    """Returns the hinge loss for preds %labels"""
    try:
        assert len(preds) == len(labels)
    except AssertionError:
        print('preds and labels have different length')
    n_samples = len(preds)
    return (1.0*np.power(np.linalg.norm(preds-labels),2)) / n_samples

## 0/1 loss
def ls_binary(preds, labels):
    """Returns the 0/1 loss for preds %labels"""
    preds = np.sign(preds)
    return ls_squared(0.5*preds, 0.5*labels)

## Hinge loss
def ls_hinge(preds, labels):
    """Returns the hinge loss for preds %labels"""
    try:
        assert len(preds) == len(labels)
    except AssertionError:
        print('preds and labels have different length')
    n_samples = len(preds)
    return np.mean(
        np.maximum(
            np.ones((n_samples,1)) - preds*labels,
            np.zeros((n_samples,1))
        )
    )

## Building metrics for reporting on performance
class Metric():
    def __init__(self, name, measure):
        self.name = name
        self.measure = measure
        
m_binary = Metric('Match rate', lambda preds,labels: 1 - ls_binary(preds,labels))

### Kernel method parent & kernel SVM

Throughout the challenge we will need to use different kernel methods, which will share some attributes and methods. I will thus create an "abstract" class kernelMethod:

In [ ]:
## Kernel methods parent class
class kernelMethod():
    def __init__(self):
        return 0

The first thing I will try out is a kernel SVM method:

In [ ]:
## Implementing a kernel SVM method:
class kernelSVM(kernelMethod):
    def __init__(self, lbda=0.1, solver='cvxopt'):
        self.lbda = lbda
        self.solver = solver
        self.data = None
        self.alpha = None
        self.kernel_fct = None
    
    def format_labels(self, labels):
        try:
            assert len(np.unique(labels)) == 2
        except AssertionError:
            print('Error: Labels provided are not binary')
        lm,lM = np.min(labels), np.max(labels)
        l = (labels==lM).astype(int) - (labels==lm).astype(int)
        return l
    
    def run(self, data, labels, kernel_fct):
        """Trains the kernel SVM on data and labels"""
        n_samples = labels.shape[0]
        # Turning labels into ±1
        labels = self.format_labels(labels)
        # Binding kernel fct and data as attribute for further predictions
        self.kernel_fct = kernel_fct
        self.data = data
        # Building matrices for solving dual problem
        print('Building kernel matrix from {0:d} samples...'.format(n_samples))
        tick = time.time()
        K = build_kernel(data, data, kernel_fct)
        print('...done in {0:.2f}s'.format(time.time()-tick))
        d = np.diag(labels)
        P = matrix((-1.0/(2*self.lbda))*d*K*d, tc='d')
        q = matrix(np.ones((n_samples,1)), tc='d')
        G1 = -np.eye(n_samples)
        G2 = np.eye(n_samples)
        G = matrix(np.vstack((G1,G2)), tc='d')
        h1 = np.zeros((n_samples,1))
        h2 = (1.0/n_samples)*np.ones((n_samples,1))
        h = matrix(np.vstack((h1,h2)), tc='d')
        # Construct the QP, invoke solver
        sol = solvers.qp(P,q,G,h)
        # Extract optimal value and solution
        dual = sol['x']
        # Solving dual problem via solver
        self.alpha = (1.0/(2*self.lbda))*(d @ dual)
    
    def predict(self, data):
        """Predict labels for data"""
        try:
            assert self.alpha is not None
            assert self.kernel_fct is not None
        except AssertionError:
            print('Error: No successful training recorded')
        # Build sv alpha and sv K(x_i(new_data), x_j(ref))
        sv_ind = np.nonzero(self.alpha)[0]
        sv_alpha = self.alpha[sv_ind]
        sv_K = build_kernel(data, self.data[sv_ind], self.kernel_fct)
        # Use supvec alpha and supvec K to compute predictions
        return sv_K @ sv_alpha
    
    def assess(self, data, labels, metrics):
        """Provides the performance of the algorithm on some test data"""
        try:
            assert len(data) == len(labels)
        except AssertionError:
            print('Error: Data and labels have different length')
        labels = self.format_labels(labels).reshape((len(labels),1))
        preds = self.predict(data)
        m = {}
        if metrics is not None:
            for metric in metrics:
                m[metric.name] = metric.measure(preds, labels)
        return preds, m

Now let's try out our kernel SVM:

In [ ]:
## Method definition
lbda = 0.0005
kSVM = kernelSVM(lbda)

In [ ]:
## Training SVM + performance assessment on training data
kSVM.run(tr0['Sequence'], tr0['Bound'], levenshtein_distance)
preds_kSVM_tr0, perf_kSVM_tr0 = kSVM.assess(tr0['Sequence'], tr0['Bound'], metrics=[m_binary])
print('Training dataset {0:d}: {1:s}: {2:.1f}%'.format(0, list(perf_kSVM_tr0.keys())[0], 100*list(perf_kSVM_tr0.values())[0]))
kSVM_te0_raw = np.sign(kSVM.predict(te0['Sequence'])).astype(int)

In [ ]:
## Training SVM + performance assessment on training data
kSVM.run(tr1['Sequence'], tr1['Bound'], levenshtein_distance)
preds_kSVM_tr1, perf_kSVM_tr1 = kSVM.assess(tr1['Sequence'], tr1['Bound'], metrics=[m_binary])
print('Training dataset {0:d}: {1:s}: {2:.1f}%'.format(1, list(perf_kSVM_tr1.keys())[0], 100*list(perf_kSVM_tr1.values())[0]))
kSVM_te1_raw = np.sign(kSVM.predict(te1['Sequence'])).astype(int)

In [ ]:
## Training SVM + performance assessment on training data
kSVM.run(tr2['Sequence'], tr2['Bound'], levenshtein_distance)
preds_kSVM_tr2, perf_kSVM_tr2 = kSVM.assess(tr2['Sequence'], tr2['Bound'], metrics=[m_binary])
print('Training dataset {0:d}: {1:s}: {2:.1f}%'.format(2, list(perf_kSVM_tr2.keys())[0], 100*list(perf_kSVM_tr2.values())[0]))
kSVM_te2_raw = np.sign(kSVM.predict(te2['Sequence'])).astype(int)

<b>Current performance rate</b>: ??% on training set, ?? on test set

<b>Next steps - Results</b>:
- What is the reason for such a poor performance rate, even on the training data?
- If this is due to Hamming being mostly irrelevant, implement the Levenshtein distance and retry with this new kernel

<b>Next steps - Computing speed</b>:
- Find a way to vectorize the kernel matrix computation

In [ ]:
def format_preds(preds):
    return (0.5*(1+np.sign(preds))).astype(int)

In [ ]:
## Predictions on test data
kSVM_te0 = pd.DataFrame(
    data = format_preds(kSVM_te0_raw),
    columns = ['Prediction'])

kSVM_te1 = pd.DataFrame(
    data = format_preds(kSVM_te1_raw),
    columns = ['Prediction'])
kSVM_te1.index = kSVM_te1.index + 1000

kSVM_te2 = pd.DataFrame(
    data = format_preds(kSVM_te2_raw),
    columns = ['Prediction'])
kSVM_te2.index = kSVM_te2.index + 2000

frames = [kSVM_te0, kSVM_te1, kSVM_te2]
kSVM_te = pd.concat(frames)
kSVM_te.index.rename('ID')

kSVM_te.to_csv('predictions/kSVM_te.csv')